# # Classifiers comparison on texts with naive Bayes assumption

In this session of laboratory we compare two models for categorical data probabilistic modeling: 
1. multivariate Bernoulli 
2. multinomial on a dataset 

We adopt a dataset on Twitter messages labelled with emotions (Joy vs Sadness).

The following program shows the loading of the data from a file.

Data are loaded into a matrix X adopting a sparse matrix representation, in order to save space and time.
Sparse matrix representation (in the csr format) represents in three "parallel" arrays the value of the matrix cells that are different from zero and the indices of those matrix cells.
The arrays are called: 
- data
- row
- col

- data[i] stores the value of the matrix cell #i whose indexes are contained in row[i] and col[i] 
- row[i] stores the index of the row in the matrix of the cell #i, 
- col[i] stores the index of the column of the cell #i.


The data file is in csv format.
Any Twitter message has been preprocessed by a Natural Language pipeline which eliminated stop words and substituted the interesting document elements with an integer identifier.  
The interesting document elements might be words, emoji or emoticons. The elements could be repeated in the same document and are uniquely identified in the documents by the same integer number (named "element_id" in the program). This "element_id" number will be used as the index of the column of the data matrix, for the purposes of storage of data.

Each row of the CSV file reports the content of a document (a Twitter message).It is formed as a list of integer number pairs, followed by a string which is the label of the document ("Joy" or "Sadness").
The first number of the pair is the identifier of a document element (the "element_id"); 
the second number of the pair is the count (frequency) of that element in that document.

The dataset has:

tot_n_docs (or rows in the file) =n_rows=11981

n_features (total number of distinct words in the corpus)=11288



The following program reads the data file and loads in a sparse way the matrix using the scipy.sparse library

In [1]:

from numpy import ndarray, zeros
import numpy as np
import scipy
from sklearn import metrics
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_roc_curve
from google.colab import drive
drive.mount('/content/drive')

class_labels = ["Joy","Sadness"]
n_features=11288 # number of columns in the matrix = number of features (distinct elements in the documents)
n_rows=11981 # number rows of the matrix
n_elements=71474 # number of the existing values in the matrix (not empty, to be loaded in the matrix in a sparse way)

file_name="/content/drive/MyDrive/Colab Notebooks/joy_sadness6000.txt"

# declare the row and col arrays with the indexes of the matrix cells (non empty) to be loaded from file
# they are needed because the matrix is sparse and we load in the matrix only the elements which are present
row=np.empty(n_elements, dtype=int)
col=np.empty(n_elements, dtype=int)
data=np.empty(n_elements, dtype=int)

row_n=0 # number of current row to be read and managed
cur_el=0 # position in the three arrays: row, col and data
twitter_labels=[] # list of class labels (target array) of the documents (twitter) that will be read from the input file
twitter_target=[] # list of 0/1 for class labels
with open(file_name, "r") as fi:
    for line in fi:
        el_list=line.split(',')  # list of integers read from a row of the file
        l=len(el_list)
        class_name=el_list[l-1].strip()  # I grab the last element in the list which is the class label# eliminate the '\n'
        twitter_labels.append(class_name)
        # twitter_labels contains the labels (Joy/Sadness); twitter_target contains 0/1 for the respective labels
        if (class_name==class_labels[0]):
           twitter_target.append(0)
        else:
           twitter_target.append(1)
        i=0 # I start reading all the doc elements from the beginning of the list
        while(i<(l-1)):
            element_id=int(el_list[i]) # identifier of the element in the document equivalent to the column index
            element_id=element_id-1 # the index starts from 0 (the read id starts from 1)
            i=i+1
            value_cell=int(el_list[i]) # make access to the following value in the file which is the count of the element in the documento 
            i=i+1
            row[cur_el]=row_n # load the data in the three arrays: the first two are the row and col indexes; the last one is the matrix cell value
            col[cur_el]=element_id
            data[cur_el]=value_cell
            cur_el=cur_el+1
        row_n=row_n+1
fi.close
print("final n_row="+str(row))
# loads the matrix by means of the indexes and the values in the three arrays just filled
twitter_data=csr_matrix((data, (row, col)), shape=(n_rows, n_features)).toarray()
print("resulting matrix:")
print(twitter_data)
print(twitter_labels)
print(twitter_target)

Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Esercizio 1
Write a program in the following cell that splits the data matrix in training and test set (by random selection) and predicts the class (Joy/Sadness) of the messages on the basis of the words. 
Consider the two possible models:
multivariate Bernoulli and multinomial Bernoulli.
Find the accuracy of the models and test is the observed differences are significant.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(twitter_data, twitter_target, test_size=0.33, random_state=42)
#Multivariate Bernoulli
clf_multivariate = BernoulliNB()
xTrainNormalize = (X_train >= 1) * 1
xTestNormalize = (X_test >= 1) * 1

clf_multivariate.fit(xTrainNormalize, y_train)
predicted_1 = clf_multivariate.predict(xTestNormalize)
accuracy_1 = accuracy_score(y_test,predicted_1) # TP+FP/TP+TN+FP+FN
print("multivariate accuracy: " + str(accuracy_1))
plot_roc_curve(clf_multivariate,xTestNormalize,y_test)



In [ ]:
#Multinomial 
clf_multinomial = MultinomialNB()
clf_multinomial.fit(X_train, y_train)
predicted_2 = clf_multinomial.predict(X_test)
accuracy_2 = accuracy_score(y_test,predicted_2)
print("multinomiale accuracy: " + str(accuracy_2))
plot_roc_curve(clf_multinomial,X_test,y_test)

 T Student Test con $\alpha = 0.05$

Misura se gli score differiscono in maniera significativa tra i modelli considerato che sono generati a partire dallo stesso campione

Ipotesi Nulla: I due score hanno valori attesi identici.

In [ ]:
from tabulate import tabulate
import math
from sklearn.model_selection import cross_val_score
from scipy import stats

scores_MVB = cross_val_score(clf_multivariate, twitter_data, twitter_target, cv=10)
scores_MNB = cross_val_score(clf_multinomial, twitter_data, twitter_target, cv=10)

print("Multivariate accuracies: ")
print(scores_MVB)
print("Multinomial accuracies: ")
print(scores_MNB)

diff = scores_MVB - scores_MNB
print("Difference: ")
print(diff)
avg = np.average(diff) # average of difference vector

alpha = 0.05
critical_value = 2.262 # critical value alpha = 0.5 (two tails) and degrees freedom (n - 1) = 10 - 1= 9
t_value = stats.ttest_rel(scores_MVB, scores_MNB)#related, perchè i dataset sono uguali, e verificare se i modelli sono statisticamente significativi
print(tabulate([['critic_value',critical_value],['t_test',t_value]], headers=['Scores', 'MVB - MNB']))

print("\n\nP-value Null Hypothesis: 2 related or repeated alghoritms have identical average (expected) values.")

if t_value.pvalue < alpha:
    print("\nReject the null hypothesis because p-value is smaller than alpha.")
else:
    print("\nCan't reject Null hypothesis.")

# Observations on the experimental test:
# Since the critical region (-2.262, 2.262) and t_test = 3.2
# if $pvalue > alfa \rightarrow$ non rejected $\Rightarrow 0.01 < 0.5$ reject $H_{0}$